In [1]:
# general packages
import random
import warnings
import seaborn as sns


from dask import dataframe as dd
from dask.distributed import Client

warnings.simplefilter('ignore')
sns.set_style('whitegrid')

In [2]:
def readHugeCsvFileAsDataFrame(file_path):
    dask_df = dd.read_csv(file_path, low_memory=False, blocksize=50000)  # 50MB chunk-size
    return dask_df.compute()

In [3]:
path_to_file = 'D:\\ML_Sessional\\train.csv'
scheduler = Client()
df = scheduler.submit(readHugeCsvFileAsDataFrame, path_to_file)
workingDataFrame = df.result()

In [4]:
from sklearn.utils import shuffle
workingDataFrame = shuffle(workingDataFrame)
workingDataFrame.reset_index(inplace=True, drop=True)

adding a new column `kfold` and giving all value as `-1`

In [5]:
workingDataFrame.loc[:,'kfold'] = -1
workingDataFrame

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme,kfold
0,Train_103005,162,3,0,হ্মী,-1
1,Train_68987,27,0,0,গ্ম,-1
2,Train_83856,115,7,0,মে,-1
3,Train_56343,13,1,6,ক্র্যা,-1
4,Train_160005,72,9,2,র্দো,-1
...,...,...,...,...,...,...
200835,Train_188223,14,7,0,ক্কে,-1
200836,Train_70721,151,0,0,স্থ,-1
200837,Train_15165,77,4,0,দ্ভু,-1
200838,Train_136487,161,0,0,হ্ব,-1


In [7]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

x = workingDataFrame.image_id.values
y = workingDataFrame[['grapheme_root','vowel_diacritic','consonant_diacritic']].values

mskf = MultilabelStratifiedKFold(n_splits=5)

for fold, (train,validation) in enumerate(mskf.split(x,y)):
    # print("train : ",train," Validation : ",validation)
    workingDataFrame.loc[validation,"kfold"] = fold

# print(workingDataFrame.kfold.value_counts())
workingDataFrame.to_csv('../input/train_folds.csv',index=False)